## Requesting Annotations by Reference Designator

In this example we show how to request all annotations made for the Cabled Array Oregon Slope Base (RS01SBPS) Shallow Profiler (SF01A) Nitrate (4A-NUTNRA101) sensor. The same technique applies to all instruments (Reference Designators) on the OOI.

In [1]:
import requests
import json
import netCDF4 as nc
import datetime
import pandas as pd

Below we have entered credential for a default user. These should work, unless they have been reset.  
Alternatively, you can login in at https://ooinet.oceanobservatories.org/ and obtain a API username and API token under your profile (top right corner).

In [2]:
username = 'OOIAPI-D8S960UXPK4K03'
token = 'IXL48EQ2XY'

Specify the reference designator you are looking for. For further information on finding reference designators for other instruments and be found via the data team portal at http://ooi.visualocean.net.

In [3]:
refdes = 'RS01SBPS-SF01A-4A-NUTNRA101'

The next step specifies to return all annotations between 2012 and now. Time stamps are converted from a string to milliseconds since 1970-01-01.

In [4]:
beginDT = int(nc.date2num(datetime.datetime.strptime("2012-01-01T01:00:01Z",'%Y-%m-%dT%H:%M:%SZ'),'seconds since 1970-01-01')*1000)
endDT = int(nc.date2num(datetime.datetime.utcnow(),'seconds since 1970-01-01')*1000)

Now we are ready to build the data request url and send the request.

In [5]:
anno_base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12580/anno/find?' # base url and port for annotations

params = { # define parameters
    'beginDT':beginDT,
    'endDT':endDT,
    'refdes':refdes,
}

r = requests.get(anno_base_url, params=params,auth=(username, token)) # send data request

data = pd.read_json(json.dumps(r.json())) # convert json response to pandas dataframe

As mentioned above, time stamps from the annotations API are handled in milliseconds since 1970-01-01. Let's create a function to convert them back to human readable time stamps and convert the millisecond time stamps in the data frame.

In [6]:
def convert_time(time_stamp):
    try: 
        time_stamp = (int(time_stamp)) / 1000
        time_stamp = nc.num2date(time_stamp,'seconds since 1970-01-01')
    except:
        pass
    return time_stamp

# convert time stamps
data['beginDT'] = data['beginDT'].apply(convert_time)
data['endDT'] = data['endDT'].apply(convert_time)

Now, let's select and print all the annotations that were made at the parameter level, by excluding any empty parameter level annotations. In this case, there is only one annotation that meets this criteria.

In [7]:
data.sort_values('annotation')

,@class,annotation,beginDT,endDT,exclusionFlag,id,method,node,parameters,qcFlag,sensor,source,stream,subsite
1,.AnnotationRecord,A brief violent storm caused loss of power to ...,2018-01-19 03:00:01.000,2018-01-19 19:00:01.000,False,319,None,None,[],not_operational,None,friedrich.knuth@rutgers.edu,None,RS01SBPS
13,.AnnotationRecord,Note: Corvalis data center lost power. Possibl...,2016-12-17 18:00:00.000,2016-12-17 19:00:00.000,False,1281,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
12,.AnnotationRecord,Note: Fiber break between Portland and Pacific...,2016-05-20 16:33:00.000,2016-05-20 18:04:00.000,False,1274,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
11,.AnnotationRecord,Note: Fiber break between Portland and Seattle...,2016-03-10 23:06:00.000,2016-03-11 09:30:00.000,False,1268,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
14,.AnnotationRecord,Note: Fiber break in Portland due to train cra...,2016-12-22 01:50:00.000,2016-12-23 12:44:00.000,False,1282,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
8,.AnnotationRecord,Note: Four 1-minute outages between Portland a...,2016-01-07 06:10:00.000,2016-01-07 06:52:00.000,False,1266,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
5,.AnnotationRecord,Note: Intermittent partial data loss due to st...,2015-01-31 00:00:00.000,2015-02-04 00:00:00.000,False,1251,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
16,.AnnotationRecord,Note: Lightning strike in Pacific City led to ...,2017-01-09 18:32:00.000,2017-01-09 21:30:00.000,False,1286,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
7,.AnnotationRecord,Note: Network issues due to fire that damaged ...,2015-06-13 00:00:00.000,2015-06-15 16:30:00.000,False,1259,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
15,.AnnotationRecord,Note: Network outage during major Seattle util...,2017-01-08 19:58:00.000,2017-01-08 21:41:00.000,False,1283,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS


In [8]:
param_anno = data[data['parameters'].str.len() != 0]

for i in range(len((param_anno['annotation'].values))):
    print('start time:', param_anno['beginDT'].iloc[i])
    print('end time:', param_anno['endDT'].iloc[i], '\n')
    print('affected parameters:', param_anno['parameters'].iloc[i], '\n')
    print(param_anno['annotation'].iloc[i],'\n')

start time: 2015-07-08 22:22:01.974000
end time: 2016-02-05 02:00:11.695000 

affected parameters: [2320, 315] 

This instrument was deployed with onboard processing set to freshwater mode leading to an invalid L1 Dissolved Nitrate Concentration data product. The L2 version of the data product is unaffected by that onboard setting as it is calculated using the L0 data with salinity and temperature from a co-located CTD. Redmine: 8659 

